This notebook generates a tidal forcing for 2D domain in NEMO 3.6

Plan: Use M2 and K1 consituents at JdF from 3D domain.Only U. Average across boundary.

Increase currents by factor of 2. Horizontal constriction in sill area is missing so need to artificially increase currents.


In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import os

from salishsea_tools import tidetools,nc_tools

%matplotlib inline

# Tide Data

First, quickly look at tide data for 3D model

In [2]:
T={}; U={}; V={}

path = '/data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/tides/'

for t in ['M2','K1']:
    fname = 'SalishSea2_Flux_RC13_west_tide_{}_grid_{}.nc'.format(t,'T')
    T[t]=nc.Dataset(os.path.join(path,fname))
    fname = 'SalishSea2_Flux_RC13_west_tide_{}_grid_{}.nc'.format(t,'U')
    U[t]=nc.Dataset(os.path.join(path,fname))
    fname = 'SalishSea2_Flux_RC13_west_tide_{}_grid_{}.nc'.format(t,'V')
    V[t]=nc.Dataset(os.path.join(path,fname))



In [3]:
 nc_tools.show_variables(U['K1'])

[u'xb', u'yb', u'nbidta', u'nbjdta', u'nbrdta', u'u1', u'u2']


Look at amp of U/V

In [4]:
u1={}; u2={}; v1={}; v2={}; t1={}; t2={}

for t in ['K1','M2']:
    u1[t] = U[t].variables['u1']
    u2[t] = U[t].variables['u2']
    
    v1[t] = V[t].variables['v1']
    v2[t] = V[t].variables['v2']
    
    t1[t] = T[t].variables['z1']
    t2[t] = T[t].variables['z2']

In [5]:
import math

In [6]:
def average_amp_phase(x1,x2):
    temp = np.sqrt(x1[:]**2 + x2[:]**2); temp = np.ma.array(temp,mask=temp==0)
    amp = np.mean(temp)
    pha=[]
    for i in range(0,x1.shape[1]):
       pha.append(math.atan2(x2[0,i],x1[0,i]))
    pha=np.ma.array(pha,mask=pha==0)
    pha=np.mean(pha)
    
    return amp, pha

In [7]:
Uamp={}; Vamp={}; Tamp={}; Upha={}; Vpha={}; Tpha={};
for t in ['K1','M2']:
    Uamp[t], Upha[t]= average_amp_phase(u1[t],u2[t])
    Vamp[t], Vpha[t]= average_amp_phase(v1[t],v2[t])
    Tamp[t], Tpha[t]= average_amp_phase(t1[t],t2[t])

In [8]:
u1['K1'][:].shape

(1, 87)

In [9]:
for t in ['K1','M2']:
    print t
    print Uamp[t], Vamp[t], Tamp[t]
    print Upha[t], Vpha[t], Tpha[t]

K1
0.186448183927 0.124745784662 0.402432726575
0.565605422043 0.183389151205 -1.87524389342
M2
0.293222130119 0.181446491144 0.822501739898
-1.38122753203 1.69071899544 -2.09972927195


In [10]:
print t1['K1'].shape
print u1['K1'].shape
print v1['K1'].shape

(1, 87)
(1, 87)
(1, 88)


I think I will just use the U components for the velcoity in this very simplified model. Average along the whole boundary.

# Construction of 2D forcing

Average the phase and amplitude and write to the tide components.

In [11]:
u_factor = 1# factor for increasing u

u2dZ1={}; u2dZ2={}; v2dZ1={}; v2dZ2={}; t2dZ1={}; t2dZ2={}

for t in ['K1','M2']:
    u2dZ1[t] = u_factor*Uamp[t]*math.cos(Upha[t])*np.ones((1,10))
    u2dZ2[t] = u_factor*Uamp[t]*math.sin(Upha[t])*np.ones((1,10))
    
    v2dZ1[t] = np.zeros((1,10))
    v2dZ2[t] = np.zeros((1,10))
    
    t2dZ1[t] = Tamp[t]*np.cos(Tpha[t])*np.ones((1,10))
    t2dZ2[t] = Tamp[t]*np.sin(Tpha[t])*np.ones((1,10))

# Create netCDF

In [14]:
def creat_netcdf(constituent, tidevar, Z1, Z2):   
    nemo = nc.Dataset('../Salish2D_3.6_'+constituent+'_grid_'+tidevar+'.nc', 'w', zlib=True)

    #start and end points
    lengthi=10

    # dataset attributes
    nc_tools.init_dataset_attrs(
        nemo,
        title='Tidal Boundary Conditions 2D domain',
        notebook_name='Generate_2D_tides-NEMO3.6',
        nc_filepath='/data/nsoontie/MEOPAR/2Ddomain/Salish2D_'+constituent+'_grid_'+tidevar+'.nc',
        comment='based on average values across mouth of JdF')

    # dimensions (only need x and y, don't need depth or time_counter)
    nemo.createDimension('xb', lengthi)
    nemo.createDimension('yb', 1)

    # variables
    # nbidta, ndjdta, ndrdta
    nbidta = nemo.createVariable('nbidta', 'int32' , ('yb','xb'))
    nbidta.long_name = 'i grid position'
    nbidta.units = 1
    nbjdta = nemo.createVariable('nbjdta', 'int32' , ('yb','xb'))
    nbjdta.long_name = 'j grid position'
    nbjdta.units = 1
    nbrdta = nemo.createVariable('nbrdta', 'int32' , ('yb','xb'))
    nbrdta.long_name = 'position from boundary'
    nbrdta.units = 1
    print nbidta.shape
    # add in the counter around the boundary (taken from Susan's code in Prepare Tide Files)
    xb = nemo.createVariable('xb', 'int32', ('xb',),zlib=True)
    xb.units = 'non dim'
    xb.long_name = 'counter around boundary'
    yb = nemo.createVariable('yb', 'int32', ('yb',),zlib=True)
    yb.units = 'non dim'
    yb.long_name = 'counter along boundary'
    yb[0] = 1
    xb[:] = np.arange(0,lengthi) +1

    # values
    # nbidta, nbjdta
    nbidta[:] = 1
    nbjdta[:] = np.arange(0,lengthi) +1
    nbrdta[:] = 1

    if tidevar=='T':
        z1 = nemo.createVariable('z1','float32',('yb','xb'),zlib=True)
        z1.units = 'm'
        z1.long_name = 'tidal elevation: cosine'
        z2 = nemo.createVariable('z2','float32',('yb','xb'),zlib=True)
        z2.units = 'm'
        z2.long_name = 'tidal elevation: sine'
        z1[0,:] = Z1[0,:]
        z2[0,:] = Z2[0,:]

    if tidevar=='U':
        u1 = nemo.createVariable('u1','float32',('yb','xb'),zlib=True)
        u1.units = 'm'
        u1.long_name = 'tidal x-velocity: cosine'
        u2 = nemo.createVariable('u2','float32',('yb','xb'),zlib=True)
        u2.units = 'm'
        u2.long_name = 'tidal x-velocity: sine'
        u1[0,:] = Z1[0,:]
        u2[0,:] = Z2[0,:]

    if tidevar=='V':
        v1 = nemo.createVariable('v1','float32',('yb','xb'),zlib=True)
        v1.units = 'm'
        v1.long_name = 'tidal y-velocity: cosine'
        v2 = nemo.createVariable('v2','float32',('yb','xb'),zlib=True)
        v2.units = 'm'
        v2.long_name = 'tidal y-velocity: sine'
        v1[0,:] = Z1[0,:]
        v2[0,:] = Z2[0,:]

    nc_tools.check_dataset_attrs(nemo)
    nemo.close()

In [15]:
varis=['T','U','V']
for t in['M2','K1']:
    Z1s=[t2dZ1[t],u2dZ1[t],v2dZ1[t]];
    Z2s=[t2dZ2[t],u2dZ2[t],v2dZ2[t]];
    for var,Z1,Z2 in zip(varis,Z1s,Z2s):
        creat_netcdf(t, var, Z1, Z2)

file format: NETCDF4
Conventions: CF-1.6
title: Tidal Boundary Conditions 2D domain
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/2d-domain/src/tip/Generate_2D_tides-NEMO3.6.ipynb
references: REQUIRED
history: [2015-08-03 15:03:44] Created netCDF4 zlib=True dataset.
comment: based on average values across mouth of JdF
(1, 10)
Missing value for dataset attribute: references
file format: NETCDF4
Conventions: CF-1.6
title: Tidal Boundary Conditions 2D domain
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/2d-domain/src/tip/Generate_2D_tides-NEMO3.6.ipynb
references: REQUIRED
history: [2015-08-03 15:03:44] Created netCDF4 zlib=True dataset.
comment: based on average values across mouth of JdF
(1, 10)
Missing value for dataset attribute: references
file format: NETCDF4
Conventions: CF-1.6
title: Tidal Boundary Conditions 2D domai